In [1]:
import pandas as pd 
import numpy as np 
from collections import defaultdict

In [5]:
import pandas as pd
from collections import defaultdict

# Read the dataset
data = pd.read_csv('Pure_Stat.csv')

# Process the data
fighters = defaultdict(lambda: defaultdict(int))

for _, row in data.iterrows():
    for f, w in [('1', 'Fighter 1'), ('2', 'Fighter 2')]:
        fighter = row[w]
        fighters[fighter]['Weight Class'] = row['Weight Class']
        fighters[fighter]['Total Fights'] += 1
        fighters[fighter]['Total Strikes Thrown'] += row[f'F{f} Total Strikes']
        fighters[fighter]['Total Sig Strikes'] += row[f'F{f} Sig Strikes']
        fighters[fighter]['Total Time Elapsed'] += row['Time Elapsed']
        fighters[fighter]['Total Takedown Attempt'] += row[f'F{f} TDA']
        fighters[fighter]['Total Takedown Landed'] += row[f'F{f} TDL']
        fighters[fighter]['Total Knockdowns'] += row[f'F{f} KD']

        if row['Winner'] == f'F{f}':
            fighters[fighter]['Wins'] += 1
            if row['Decision'] == 1:
                fighters[fighter]['Decision Wins'] += 1
            if row['Submission'] == 1:
                fighters[fighter]['Submission Wins'] += 1
            if row['KO/TKO'] == 1:
                fighters[fighter]['KO/TKO Wins'] += 1
        else:
            fighters[fighter]['Losses'] += 1
            if row['Decision'] == 1:
                fighters[fighter]['Decision Losses'] += 1
            if row['Submission'] == 1:
                fighters[fighter]['Submission Losses'] += 1
            if row['KO/TKO'] == 1:
                fighters[fighter]['KO/TKO Losses'] += 1

# Make sure that there are no blank values for wins and losses
for fighter in fighters:
    if 'Wins' not in fighters[fighter]:
        fighters[fighter]['Wins'] = 0
    if 'Losses' not in fighters[fighter]:
        fighters[fighter]['Losses'] = 0
    if 'Decision Wins' not in fighters[fighter]:
        fighters[fighter]['Decision Wins'] = 0

    # Calculate Strikes Thrown Per Minute
    strikes_per_minute = (fighters[fighter]['Total Strikes Thrown'] * 60) / fighters[fighter]['Total Time Elapsed']
    fighters[fighter]['Strikes Thrown Per Minute'] = strikes_per_minute
    # Calculate Sig Strikes Landed Per Minute
    sig_strikes_per_minute = (fighters[fighter]['Total Sig Strikes'] * 60) / fighters[fighter]['Total Time Elapsed']
    fighters[fighter]['Sig Strikes Per Minute'] = sig_strikes_per_minute
    # Calculate TDA Per Minute
    tda_per_minute = (fighters[fighter]['Total Takedown Attempt'] * 60) / fighters[fighter]['Total Time Elapsed']
    fighters[fighter]['TDA/Min'] = tda_per_minute
    # Calculate TDL Per Minute
    tdl_per_minute = (fighters[fighter]['Total Takedown Landed'] * 60) / fighters[fighter]['Total Time Elapsed']
    fighters[fighter]['TDL/Min'] = tdl_per_minute
    # Calculate KD Per Minute
    kd_per_minute = (fighters[fighter]['Total Knockdowns'] * 60) / fighters[fighter]['Total Time Elapsed']
    fighters[fighter]['KD/Min'] = kd_per_minute

# Create a new dataframe and save it as a CSV
fighters_df = pd.DataFrame(fighters).transpose()
fighters_df.to_csv('fighters.csv', index_label='Name')


In [3]:
import pandas as pd

# Read the dataset
data = pd.read_csv('Pure_Stat.csv')

# Group by weight class
grouped_data = data.groupby('Weight Class').agg({
    'Date': 'count',
    'Time Elapsed': 'sum',
    'F1 Total Strikes': 'sum',
    'F2 Total Strikes': 'sum',
    'F1 Sig Strikes': 'sum',
    'F2 Sig Strikes': 'sum',
    'F1 TDA': 'sum',
    'F2 TDA': 'sum',
    'F1 TDL': 'sum',
    'F2 TDL': 'sum',
    'F1 KD': 'sum',
    'F2 KD': 'sum',
    'Decision': 'sum',
    'KO/TKO': 'sum',
    'Submission': 'sum',
}).reset_index()

# Calculate totals for strikes, TDA, TDL, and KD
grouped_data['Total Strikes'] = grouped_data['F1 Total Strikes'] + grouped_data['F2 Total Strikes']
grouped_data['Total Sig Strikes'] = grouped_data['F1 Sig Strikes'] + grouped_data['F2 Sig Strikes']
grouped_data['Total TDA'] = grouped_data['F1 TDA'] + grouped_data['F2 TDA']
grouped_data['Total TDL'] = grouped_data['F1 TDL'] + grouped_data['F2 TDL']
grouped_data['Total KD'] = grouped_data['F1 KD'] + grouped_data['F2 KD']

# Select desired columns
grouped_data = grouped_data[['Weight Class', 'Date', 'Time Elapsed', 'Total Strikes', 'Total Sig Strikes', 'Total TDA', 'Total TDL', 'Total KD', 'Decision', 'KO/TKO', 'Submission']]

# Rename columns
grouped_data.columns = ['Weight Class', 'Count of Fights', 'Total Time Elapsed', 'Total Strikes Thrown', 'Total Sig Strikes', 'Total TDA', 'Total TDL', 'Total KD', 'Total Decisions', 'Total KOs', 'Total Submissions']

# Save the grouped data as a CSV
grouped_data.to_csv('averages.csv', index=False)


In [9]:
import pandas as pd

# Load the datasets
averages = pd.read_csv("averages.csv")
fighters = pd.read_csv("fighters_altered.csv")

# Create a dictionary to store the weight class averages for easy access
wc_averages = {}
for index, row in averages.iterrows():
    wc = row['Weight Class']
    wc_averages[wc] = row

# Prepare the columns for the new DataFrame
new_columns = ["Name", "Weight Class", "Total Fights"] + [f"{stat} Difference" for stat in [
    "Strikes Thrown Per Minute",
    "Sig Strikes Per Minute",
    "TDA Per Minute",
    "TDL Per Minute",
    "KD Per Minute"
]]

# Create a new DataFrame to store the differences
differences_df = pd.DataFrame(columns=new_columns)

# Iterate through the fighters dataset and calculate the difference in stats
for index, row in fighters.iterrows():
    weight_class = row['Weight Class']
    total_fights = row['Total Fights']
    fighter_stats = {
        "Strikes Thrown Per Minute": row["Strikes Thrown Per Minute"],
        "Sig Strikes Per Minute": row["Sig Strikes Per Minute"],
        "TDA Per Minute": row["TDA/Min"],
        "TDL Per Minute": row["TDL/Min"],
        "KD Per Minute": row["KD/Min"]
    }
    wc_avg_stats = {
        "Strikes Thrown Per Minute": wc_averages[weight_class]["Avg Strike/Min"],
        "Sig Strikes Per Minute": wc_averages[weight_class]["Avg Sig Strike/Min"],
        "TDA Per Minute": wc_averages[weight_class]["Avg TDA/Min"],
        "TDL Per Minute": wc_averages[weight_class]["Avg TDL/Min"],
        "KD Per Minute": wc_averages[weight_class]["Avg KD/Min"]
    }
    
    differences = [row['Name'], weight_class, total_fights] + [fighter_value - wc_avg_stats[stat] for stat, fighter_value in fighter_stats.items()]
    differences_df = differences_df.append(pd.Series(differences, index=new_columns), ignore_index=True)

# Save the differences DataFrame to a new CSV file
differences_df.to_csv("differences.csv", index=False)


C:\Users\khoob\AppData\Local\Temp\ipykernel_5892\848462887.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  differences_df = differences_df.append(pd.Series(differences, index=new_columns), ignore_index=True)
C:\Users\khoob\AppData\Local\Temp\ipykernel_5892\848462887.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  differences_df = differences_df.append(pd.Series(differences, index=new_columns), ignore_index=True)
C:\Users\khoob\AppData\Local\Temp\ipykernel_5892\848462887.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  differences_df = differences_df.append(pd.Series(differences, index=new_columns), ignore_index=True)
C:\Users\khoob\AppData\Local\Temp\ipykernel_5892\848462887.py:45: FutureWarning: The frame.app